# Нейронные сети

Типы нейронных сетей
- Сверточные (CNN) - компьютерное зрение, классификация изображений
- Рекурентные - обработка естественного языка
- Генеративные (GAN)
- Многослойный перцептрон

In [30]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50, decode_predictions
import numpy as np

In [31]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [37]:
img = image.load_img('CAT.png', target_size=(224, 224))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
print(img_batch.shape)

(1, 224, 224, 3)


D:\Program Files (x86)\Python\Lib\site-packages\PIL\Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [38]:
model = ResNet50()
img_processed = preprocess_input(img_batch)
prediction = model.predict(img_processed)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [34]:
print(decode_predictions(prediction, top=5)[0])

[('n02110958', 'pug', np.float32(0.9969117)), ('n02108422', 'bull_mastiff', np.float32(0.0023933933)), ('n02091467', 'Norwegian_elkhound', np.float32(0.00047696714)), ('n02112137', 'chow', np.float32(4.3287368e-05)), ('n02093754', 'Border_terrier', np.float32(3.9850063e-05))]


Создадим свою нейронку, используя перенос обучения. Можно перенести слои с обобщённым знанием.

- Организация данных
- Пайплайн подготовки данных (чтение обработка, группировка в пакеты)
- Аугментация данных
- Определение модели. Заморозка коэффициентов слоёв, которые мы не хотим менять
- Обучить модель. Метрика должна стать приемлемой
- Сохранение модели

In [11]:
TRAIN_DATA_DIR = 'data1/train_data'
VALIDATION_DATA_DIR = 'data1/val_data'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64
NUM_CLASSES = 2

from tensorflow.keras.applications.mobilenet import preprocess_input, MobileNet
train_datagen = image.ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2
)

val_datagen = image.ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size = (IMG_WIDTH,IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=12345,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size = (IMG_WIDTH,IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical'
)

Found 501 images belonging to 2 classes.
Found 501 images belonging to 2 classes.


In [26]:
from tensorflow.keras.layers import (
    Input,
    Flatten,
    Dense,
    Dropout,
    GlobalAveragePooling2D
)

from tensorflow.keras.models import Model
def model_maker():
    base_model = MobileNet(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False

    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    prediction = Dense(NUM_CLASSES, activation='softmax')(custom_model)

    return Model(inputs=input, outputs=prediction)

from tensorflow.keras.optimizers import Adam
model = model_maker()
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics=['acc'])

import math

num_steps = math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE)

model.fit(
    train_generator,
    steps_per_epoch=num_steps,
    epochs=10, #ШАГ ОБУЧЕНИЯ. Простамтривается весь набор данных
    validation_data=val_generator,
    validation_steps=num_steps,
)

D:\Program Files (x86)\Python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - acc: 0.5357 - loss: 0.9111 - val_acc: 0.9401 - val_loss: 0.2134
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - acc: 0.8598 - loss: 0.3062 - val_acc: 0.9641 - val_loss: 0.1188
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - acc: 0.9253 - loss: 0.1868 - val_acc: 0.9661 - val_loss: 0.0903
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - acc: 0.9242 - loss: 0.2138 - val_acc: 0.9701 - val_loss: 0.0854
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - acc: 0.9503 - loss: 0.1184 - val_acc: 0.9721 - val_loss: 0.0757
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - acc: 0.9633 - loss: 0.1077 - val_acc: 0.9661 - val_loss: 0.0782
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - acc: 0.9513 - loss: 0.1350 - val_acc: 0.9760 - val_loss: 0.0681
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - acc: 0.9700 - loss: 0.0874 - val_acc: 0.9800 - val_loss: 0.0638
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - acc: 0.9842 - loss: 0.0659 - val_acc: 

In [28]:
print(val_generator.class_indices)
model.save('data1/model.h5')

{'cat': 0, 'dog': 1}


In [39]:
from keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model('data1/model.h5')
prediction = model.predict(img_processed)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 974ms/step
[[0.9971891  0.00281091]]
